In [1]:
import pandas as pd
import datetime
import plotly.express as px
import warnings

warnings.filterwarnings("ignore")

In [2]:
import numpy
print(numpy.__version__)

2.1.3


In [3]:
res = pd.read_csv("scrapres/res1.csv", sep="|", index_col=0)
meta = pd.read_csv("scrapres/meta.csv", index_col=0)
display(res.head())
meta.head()

,cafe,post_id,post_link,day_posted,n_likes,title,first_comments
0,https://www.instagram.com/abrasileirachiado/,0,https://www.instagram.com/p/DI3YwV-PtK0/,1 day ago,79,"António de Oliveira Salazar, presidente do Con...",[]
1,https://www.instagram.com/abrasileirachiado/,1,https://www.instagram.com/p/DItuzHaB82n/,5 days ago,21,“Todos sabemos que a arte não é verdade. A art...,[]
2,https://www.instagram.com/abrasileirachiado/,2,https://www.instagram.com/p/DIjpVnwhqbe/?img_i...,17 April,28,A obra de António Faria convoca o olhar do esp...,[]
3,https://www.instagram.com/abrasileirachiado/,3,https://www.instagram.com/p/DIZWO_2PmwH/,13 April,110,A Brasileira aos olhos de quem a visita.\n\n@d...,"['👏🥂🥂🥂', 'Sei bem😘💚❤👏🏻', 'Conheço e recomendo ..."
4,https://www.instagram.com/abrasileirachiado/,4,https://www.instagram.com/p/DIOnjB_hYhU/?img_i...,9 April,29,"“Uma pessoa sem vícios tem poucas virtudes.""\n...",[]


,link,title,info,n_posts,n_followers,n_followed
0,https://www.instagram.com/abrasileirachiado/,A Brasileira do Chiado | Café Histórico,🇵🇹 O mais emblemático café histórico de Lisboa...,467,"9,526",151
1,https://www.instagram.com/majestic_cafe/,Majestic Cafe,"Mais do que um café, o Majestic conta a histór...","8,825",20.3K,414
2,https://www.instagram.com/combi_coffee/,COMBI COFFEE ROASTERS,We are Combi - an independent specialty coffee...,"1,014",21.4K,0
3,https://www.instagram.com/fabricacoffeeroasters/,FÁBRICA COFFEE ROASTERS,"Sourcing, roasting, and delivering premium cof...",782,28.7K,451
4,https://www.instagram.com/cphcoffeelab/,Copenhagen Coffee Lab,Committed to coffee and bread with love and re...,961,29.9K,"1,174"


In [4]:
def data(data):
    data = data.strip()
    if len(data.split()) == 2:
        return data + " 2025"
    elif data.split()[-1] == "ago":
        if "hours" in data:
            return "27 April 2025"
        else:
            return str(27 - int(data.split()[0])) + " April 2025"
    else:
        return data

resw = res.drop(["post_id", "title", "first_comments"], axis=1).set_index("post_link")
resw["day_posted"] = resw["day_posted"].apply(data)
resw["day_posted"] = resw["day_posted"].astype("datetime64[s]")

metaw = meta.drop(["title", "info", "n_posts", "n_followed"], axis=1)
metaw.columns = ["cafe", "followers"]

In [5]:
final = resw.reset_index().merge(metaw, on="cafe", how="inner").dropna()
final

,post_link,cafe,day_posted,n_likes,followers
0,https://www.instagram.com/p/DI3YwV-PtK0/,https://www.instagram.com/abrasileirachiado/,2025-04-26,79,"9,526"
1,https://www.instagram.com/p/DItuzHaB82n/,https://www.instagram.com/abrasileirachiado/,2025-04-22,21,"9,526"
2,https://www.instagram.com/p/DIjpVnwhqbe/?img_i...,https://www.instagram.com/abrasileirachiado/,2025-04-17,28,"9,526"
3,https://www.instagram.com/p/DIZWO_2PmwH/,https://www.instagram.com/abrasileirachiado/,2025-04-13,110,"9,526"
4,https://www.instagram.com/p/DIOnjB_hYhU/?img_i...,https://www.instagram.com/abrasileirachiado/,2025-04-09,29,"9,526"
...,...,...,...,...,...
4145,https://www.instagram.com/p/CnEWICKrs3j/,https://www.instagram.com/apadariaportuguesa,2023-01-06,"1,818",65.5K
4146,https://www.instagram.com/p/CnB3C5oM32f/?img_i...,https://www.instagram.com/apadariaportuguesa,2023-01-05,294,65.5K
4147,https://www.instagram.com/p/Cm6DBvZMpKv/,https://www.instagram.com/apadariaportuguesa,2023-01-02,"1,288",65.5K
4148,https://www.instagram.com/p/Cm3y2JUIPEH/,https://www.instagram.com/apadariaportuguesa,2023-01-01,"1,906",65.5K


In [6]:
final["n_likes"] = final["n_likes"].apply(lambda x: float(str(x).replace(",", "").replace("K", ""))*1000 if "K" in str(x) else float(str(x).replace(",", "")))
final["followers"] = final["followers"].apply(lambda x: float(str(x).replace(",", "").replace("K", ""))*1000 if "K" in str(x) else float(str(x).replace(",", "")))
final["var"] = final["n_likes"]/final["followers"]
final.drop_duplicates(["cafe", "day_posted"])
final.sort_values(by="day_posted", inplace=True)

obj = px.line(final, x="day_posted", y="var", color="cafe", labels={"var": "Ratio between Likes and Followers",
                                                                    "day_posted": "Date"},
                                                                    hover_data=["post_link"])
display(obj)

final["cafe"] = final["cafe"].apply(lambda x: x if x[-1] == "/" else x + "/").apply(lambda x: x.split("/")[-2])
px.box(final, x="cafe", y="var")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([['https://www.instagram.com/p/Ch64uDiMOCI/'],
                                   ['https://www.instagram.com/p/CmyiSc8MqJm/'],
                                   ['https://www.instagram.com/p/Cm_aRCEtQH5/'],
                                   ...,
                                   ['https://www.instagram.com/p/DItZr43MOLq/'],
                                   ['https://www.instagram.com/p/DI3tFuuMMeP/'],
                                   ['https://www.instagram.com/p/DI32nbvM5_1/']], dtype=object),
              'hovertemplate': ('cafe=https://www.instagram.com' ... '{customdata[0]}<extra></extra>'),
              'legendgroup': 'https://www.instagram.com/majestic_cafe/',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'https://www.instagram.com/majestic_cafe/',
              'showlegend': True,
              'type': 'scattergl',
              'x': array(['2022-08-31T00:00:00', '2022-12-30T00:00:00', '2023-01-04T00:00:00',
                          ..., '2025-04-22T00:00:00', '2025-04-26T00:00:00',
                          '2025-04-26T00:00:00'], dtype='datetime64[s]'),
              'xaxis': 'x',
              'y': {'bdata': ('8gQq5Rttnz+fNSPib5JzP+d3DReNwW' ... 'K10xk/nOQEhaHKZT8LYDAbZ2x5Pw=='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'customdata': array([['https://www.instagram.com/p/CmbJeNrK0SZ/'],
                                   ['https://www.instagram.com/p/CnJhtXHssEy/'],
                                   ['https://www.instagram.com/p/CoCMC0QMqCM/?img_index=1'],
                                   ['https://www.instagram.com/p/CoMdC0qsNOj/?img_index=1'],
                                   ['https://www.instagram.com/p/CoUKt8KsV0r/'],
                                   ['https://www.instagram.com/p/CouAJJNsebX/?img_index=1'],
                                   ['https://www.instagram.com/p/Co4O0cos6mH/?img_index=1'],
                                   ['https://www.instagram.com/p/CpHqRISMSQ8/'],
                                   ['https://www.instagram.com/p/CpPeYuzs0na/?img_index=1'],
                                   ['https://www.instagram.com/p/CpR_hNHMVDb/?img_index=1'],
                                   ['https://www.instagram.com/p/CpUiIxYsVSd/?img_index=1'],
                                   ['https://www.instagram.com/p/Cpw2Sa7McCw/?img_index=1'],
                                   ['https://www.instagram.com/p/Cqxv2kgtJ87/?img_index=1'],
                                   ['https://www.instagram.com/p/CrAqMs2sefx/?img_index=1'],
                                   ['https://www.instagram.com/p/CrIT0qoM-B4/?img_index=1'],
                                   ['https://www.instagram.com/p/CrK0uNnM8mA/'],
                                   ['https://www.instagram.com/p/CrNgxSAoZtU/'],
                                   ['https://www.instagram.com/p/CrQAv7NsXcZ/'],
                                   ['https://www.instagram.com/p/CrVwnbHt6Xm/'],
                                   ['https://www.instagram.com/p/Crc40BIs4Pt/'],
                                   ['https://www.instagram.com/p/CrfbEE-NcJe/'],
                                   ['https://www.instagram.com/p/CrfwlQcs-x_/'],
                                   ['https://www.instagram.com/p/CriHocAsJ68/'],
                                   ['https://www.instagram.com/p/CrktoHPrczu/'],
                                   ['https://www.instagram.com/p/CrnL-hUM7LX/?img_index=1'],
                                   ['https://www.instagram.com/p/CryUqeCtdA-/'],
                                   ['https://www.instagram.com/p/Cr-e7_uMSG8/'],
                                   ['https://www.instagram.com/p/CsF_Ouoto-h/'],
                                   ['https://www.instagram.com/p/CsQZU00M-3J/'],
                                   ['https://www.instagram.com/p/

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'cafe=%{x}<br>var=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa'},
              'name': '',
              'notched': False,
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'box',
              'x': array(['majestic_cafe', 'combi_coffee', 'fabricacoffeeroasters', ...,
                          'abrasileirachiado', 'timhortons', 'timhortons'], dtype=object),
              'x0': ' ',
              'xaxis': 'x',
              'y': {'bdata': ('8gQq5Rttnz8kJz071y1rP4z0drZKHm' ... 'ecC/j7gD9qkMzlGb2DP7TbeFgewUw/'),
                    'dtype': 'f8'},
              'y0': ' ',
              'yaxis': 'y'}],
    'layout': {'boxmode': 'group',
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'cafe'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'var'}}}
})

In [7]:
final["month"] = final["day_posted"].dt.month
final["year"] = final["day_posted"].dt.year
final = final[final["year"]>2022]
a = final[["cafe", "var", "month", "year"]]
a = a.groupby(["cafe", "year", "month"])[["var"]].median()
a = a.reset_index()
a["new_date"] = a["year"].astype(str) + "-" + a["month"].astype(str) + "-01"
a["new_date"] = a["new_date"].astype("datetime64[s]")
a = a.sort_values(by="new_date")
px.line(a, x="new_date", y="var", color="cafe", labels={"var": "Median monthly ratio between Likes and Followers",
                                                        "new_date": "Date"})

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('cafe=abrasileirachiado<br>Date' ... ' Followers=%{y}<extra></extra>'),
              'legendgroup': 'abrasileirachiado',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'abrasileirachiado',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2023-01-01T00:00:00', '2023-02-01T00:00:00', '2023-03-01T00:00:00',
                          '2023-04-01T00:00:00', '2023-05-01T00:00:00', '2023-06-01T00:00:00',
                          '2023-07-01T00:00:00', '2023-08-01T00:00:00', '2023-09-01T00:00:00',
                          '2023-10-01T00:00:00', '2023-11-01T00:00:00', '2023-12-01T00:00:00',
                          '2024-01-01T00:00:00', '2024-02-01T00:00:00', '2024-03-01T00:00:00',
                          '2024-04-01T00:00:00', '2024-05-01T00:00:00', '2024-06-01T00:00:00',
                          '2024-07-01T00:00:00', '2024-08-01T00:00:00', '2024-09-01T00:00:00',
                          '2024-10-01T00:00:00', '2024-11-01T00:00:00', '2024-12-01T00:00:00',
                          '2025-01-01T00:00:00', '2025-02-01T00:00:00', '2025-03-01T00:00:00',
                          '2025-04-01T00:00:00'], dtype='datetime64[s]'),
              'xaxis': 'x',
              'y': {'bdata': ('XviJ+6ioej+5j4qqATOBP4N7cupCGX' ... 'ADRbRyP9VkxM77yXY/hgecC/j7gD8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': ('cafe=timhortons<br>Date=%{x}<b' ... ' Followers=%{y}<extra></extra>'),
              'legendgroup': 'timhortons',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'timhortons',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2023-01-01T00:00:00', '2023-02-01T00:00:00', '2023-03-01T00:00:00',
                          '2023-04-01T00:00:00', '2023-05-01T00:00:00', '2023-06-01T00:00:00',
                          '2023-07-01T00:00:00', '2023-08-01T00:00:00', '2023-09-01T00:00:00',
                          '2023-10-01T00:00:00', '2023-11-01T00:00:00', '2023-12-01T00:00:00',
                          '2024-01-01T00:00:00', '2024-02-01T00:00:00', '2024-03-01T00:00:00',
                          '2024-04-01T00:00:00', '2024-05-01T00:00:00', '2024-06-01T00:00:00',
                          '2024-07-01T00:00:00', '2024-08-01T00:00:00', '2024-09-01T00:00:00',
                          '2024-10-01T00:00:00', '2024-11-01T00:00:00', '2024-12-01T00:00:00',
                          '2025-01-01T00:00:00', '2025-02-01T00:00:00', '2025-03-01T00:00:00',
                          '2025-04-01T00:00:00'], dtype='datetime64[s]'),
              'xaxis': 'x',
              'y': {'bdata': ('VVkUg3IRZD/Md5znXZFqP0bsMnn7rn' ... 'MNgwNqPyiDv/fDrVM/xKxAaiN7cT8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': ('cafe=combi_coffee<br>Date=%{x}' ... ' Followers=%{y}<extra></extra>'),
              'legendgroup': 'combi_coffee',
              'line': {'color': '#00cc96', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'combi_coffee',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2023-01-01T00:00:00', '2023-02-01T00:00:00', '2023-03-01T00:00:00',
                          '2023-04-01T00:00:00', '2023-05-01T00:00:00', '2023-06-01T00:00:00',
                          '2023-07-01T00:00:00', '2023-08-01T00:00:00', '2023-09-01T00:00:00',
                          '2023-10-01T00:00:00', '2023-11-01T00:00:00', '2023-12-01T00:00:00',
                          '2024-02-01T00:00:00', '2024-03-01T00:00:00', '2024-0

In [8]:
final["month"] = final["day_posted"].dt.month
final["year"] = final["day_posted"].dt.year
final["trimester"] = final["month"].apply(lambda x: (x-1)//3+1)
final = final[final["year"]>2022]
a = final[["cafe", "var", "trimester", "year"]]
a = a.groupby(["cafe", "year", "trimester"])[["var"]].median()
a = a.reset_index()
a["new_date"] = a["year"].astype(str) + "-" + a["trimester"].apply(lambda x: 1 + 3*(x-1)).astype(str) + "-01"
a["new_date"] = a["new_date"].astype("datetime64[s]")
a = a.sort_values(by="new_date")
px.line(a, x="new_date", y="var", color="cafe", labels={"var": "Median trimesteral ratio between Likes and Followers",
                                                        "new_date": "Date"})

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('cafe=abrasileirachiado<br>Date' ... ' Followers=%{y}<extra></extra>'),
              'legendgroup': 'abrasileirachiado',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'abrasileirachiado',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2023-01-01T00:00:00', '2023-04-01T00:00:00', '2023-07-01T00:00:00',
                          '2023-10-01T00:00:00', '2024-01-01T00:00:00', '2024-04-01T00:00:00',
                          '2024-07-01T00:00:00', '2024-10-01T00:00:00', '2025-01-01T00:00:00',
                          '2025-04-01T00:00:00'], dtype='datetime64[s]'),
              'xaxis': 'x',
              'y': {'bdata': ('8Tn+8vVgfD+V189/gsx5Pzl1oQwPOH' ... '52uEh1P3kClluINXQ/hgecC/j7gD8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': ('cafe=starbucksptg<br>Date=%{x}' ... ' Followers=%{y}<extra></extra>'),
              'legendgroup': 'starbucksptg',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'starbucksptg',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2023-01-01T00:00:00', '2023-04-01T00:00:00', '2023-07-01T00:00:00',
                          '2023-10-01T00:00:00', '2024-01-01T00:00:00', '2024-04-01T00:00:00',
                          '2024-07-01T00:00:00', '2024-10-01T00:00:00', '2025-01-01T00:00:00',
                          '2025-04-01T00:00:00'], dtype='datetime64[s]'),
              'xaxis': 'x',
              'y': {'bdata': ('+cEC+Gm2fD/5iznKm+16P9xk8u54dI' ... 'MKpIaAP/qnZNwbAng/+p5gc+Yraz8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': ('cafe=majestic_cafe<br>Date=%{x' ... ' Followers=%{y}<extra></extra>'),
              'legendgroup': 'majestic_cafe',
              'line': {'color': '#00cc96', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'majestic_cafe',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2023-01-01T00:00:00', '2023-04-01T00:00:00', '2023-07-01T00:00:00',
                          '2023-10-01T00:00:00', '2024-01-01T00:00:00', '2024-04-01T00:00:00',
                          '2024-07-01T00:00:00', '2024-10-01T00:00:00', '2025-01-01T00:00:00',
                          '2025-04-01T00:00:00'], dtype='datetime64[s]'),
              'xaxis': 'x',
              'y': {'bdata': ('xm5kQxgFaT+ihkE/PlphP5zkBIWhym' ... '7XcNFoP7GpNOTcZ2c/t0txnnn3Yj8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': ('cafe=jeronymo_pt<br>Date=%{x}<' ... ' Followers=%{y}<extra></extra>'),
              'legendgroup': 'jeronymo_pt',
              'line': {'color': '#ab63fa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'jeronymo_pt',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2023-01-01T00:00:00', '2023-04-01T00:00:00', '2023-07-01T00:00:00',
                          '2023-10-01T00:00:00', '2024-01-01T00:00:00', '2024-04-01T00:00:00',
                          '2024-07-01T00:00:00', '2024-10-01T00:00:00', '2025-01-01T00:00:00',
                          '2025-04-01T00:00:00'], dtype='datetime64[s]'),
              'xaxis': 'x',
              'y': {'bdata': ('ferP34pUpz9afWPetP14P9CTdzh+dK' ... 'OcbV5+P1ji6u737YM/qvNrD7mseD8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': ('cafe=fabricacoffeeroasters<b

In [9]:
a = final[["cafe", "n_likes", "month", "year"]]
a = a.groupby(["cafe", "year", "month"])[["n_likes"]].median()
a = a.reset_index()
a["new_date"] = a["year"].astype(str) + "-" + a["month"].astype(str) + "-01"
a["new_date"] = a["new_date"].astype("datetime64[s]")
a = a.sort_values(by="new_date")
px.line(a, x="new_date", y="n_likes", color="cafe", labels={"n_likes": "Median Monthly Number of Likes",
                                                            "new_date": "Month"})

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('cafe=abrasileirachiado<br>Mont' ... 'r of Likes=%{y}<extra></extra>'),
              'legendgroup': 'abrasileirachiado',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'abrasileirachiado',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2023-01-01T00:00:00', '2023-02-01T00:00:00', '2023-03-01T00:00:00',
                          '2023-04-01T00:00:00', '2023-05-01T00:00:00', '2023-06-01T00:00:00',
                          '2023-07-01T00:00:00', '2023-08-01T00:00:00', '2023-09-01T00:00:00',
                          '2023-10-01T00:00:00', '2023-11-01T00:00:00', '2023-12-01T00:00:00',
                          '2024-01-01T00:00:00', '2024-02-01T00:00:00', '2024-03-01T00:00:00',
                          '2024-04-01T00:00:00', '2024-05-01T00:00:00', '2024-06-01T00:00:00',
                          '2024-07-01T00:00:00', '2024-08-01T00:00:00', '2024-09-01T00:00:00',
                          '2024-10-01T00:00:00', '2024-11-01T00:00:00', '2024-12-01T00:00:00',
                          '2025-01-01T00:00:00', '2025-02-01T00:00:00', '2025-03-01T00:00:00',
                          '2025-04-01T00:00:00'], dtype='datetime64[s]'),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAT0AAAAAAAABUQAAAAAAAgF' ... 'AAAMBFQAAAAAAAgEpAAAAAAADAU0A='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': ('cafe=timhortons<br>Month=%{x}<' ... 'r of Likes=%{y}<extra></extra>'),
              'legendgroup': 'timhortons',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'timhortons',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2023-01-01T00:00:00', '2023-02-01T00:00:00', '2023-03-01T00:00:00',
                          '2023-04-01T00:00:00', '2023-05-01T00:00:00', '2023-06-01T00:00:00',
                          '2023-07-01T00:00:00', '2023-08-01T00:00:00', '2023-09-01T00:00:00',
                          '2023-10-01T00:00:00', '2023-11-01T00:00:00', '2023-12-01T00:00:00',
                          '2024-01-01T00:00:00', '2024-02-01T00:00:00', '2024-03-01T00:00:00',
                          '2024-04-01T00:00:00', '2024-05-01T00:00:00', '2024-06-01T00:00:00',
                          '2024-07-01T00:00:00', '2024-08-01T00:00:00', '2024-09-01T00:00:00',
                          '2024-10-01T00:00:00', '2024-11-01T00:00:00', '2024-12-01T00:00:00',
                          '2025-01-01T00:00:00', '2025-02-01T00:00:00', '2025-03-01T00:00:00',
                          '2025-04-01T00:00:00'], dtype='datetime64[s]'),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAADwlEAAAAAAALibQAAAAAAAaK' ... 'AAACSbQAAAAAAAiIRAAAAAAAA9okA='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': ('cafe=combi_coffee<br>Month=%{x' ... 'r of Likes=%{y}<extra></extra>'),
              'legendgroup': 'combi_coffee',
              'line': {'color': '#00cc96', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'combi_coffee',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2023-01-01T00:00:00', '2023-02-01T00:00:00', '2023-03-01T00:00:00',
                          '2023-04-01T00:00:00', '2023-05-01T00:00:00', '2023-06-01T00:00:00',
                          '2023-07-01T00:00:00', '2023-08-01T00:00:00', '2023-09-01T00:00:00',
                          '2023-10-01T00:00:00', '2023-11-01T00:00:00', '2023-12-01T00:00:00',
                          '2024-02-01T00:00:00', '2024-03-01T00:00:00', '2024-0

In [10]:
final

,post_link,cafe,day_posted,n_likes,followers,var,month,year,trimester
237,https://www.instagram.com/p/Cm35pKzMx2q/,abrasileirachiado,2023-01-01,35.0,9526.0,0.003674,1,2023,1
2892,https://www.instagram.com/p/Cm3XRm5soUL/?img_i...,costacoffee,2023-01-01,1317.0,430000.0,0.003063,1,2023,1
4148,https://www.instagram.com/p/Cm3y2JUIPEH/,apadariaportuguesa,2023-01-01,1906.0,65500.0,0.029099,1,2023,1
4147,https://www.instagram.com/p/Cm6DBvZMpKv/,apadariaportuguesa,2023-01-02,1288.0,65500.0,0.019664,1,2023,1
3718,https://www.instagram.com/p/Cm9gEt2NYgY/,jeronymo_pt,2023-01-03,395.0,6474.0,0.061013,1,2023,1
...,...,...,...,...,...,...,...,...,...
3410,https://www.instagram.com/p/DI8jHkQqnTs/,jeronymo_pt,2025-04-26,7.0,6474.0,0.001081,4,2025,2
1807,https://www.instagram.com/p/DI5sXSNtIi2/?img_i...,cphcoffeelab,2025-04-26,86.0,29900.0,0.002876,4,2025,2
0,https://www.instagram.com/p/DI3YwV-PtK0/,abrasileirachiado,2025-04-26,79.0,9526.0,0.008293,4,2025,2
2898,https://www.instagram.com/p/DI6blnZx2SE/,timhortons,2025-04-27,5272.0,547000.0,0.009638,4,2025,2


In [11]:
costa = final[final['cafe']=='costacoffee']

def fun(x):
    if x < costa["var"].quantile(0.5):
        return -1
    #elif x < costa["var"].quantile(0.5):
    #    return 1
    elif x < costa["var"].quantile(0.75) + 1.5*(costa["var"].quantile(0.75) - costa["var"].quantile(0.25)):
        return 0
    else:
        return 1

costa['q'] = costa['var'].apply(lambda x: fun(x))
costa

,post_link,cafe,day_posted,n_likes,followers,var,month,year,trimester,q
2892,https://www.instagram.com/p/Cm3XRm5soUL/?img_i...,costacoffee,2023-01-01,1317.0,430000.0,0.003063,1,2023,1,0
2891,https://www.instagram.com/p/Cm9kKVGvuQs/,costacoffee,2023-01-03,762.0,430000.0,0.001772,1,2023,1,-1
2890,https://www.instagram.com/p/CnAeWX8hyF7/,costacoffee,2023-01-04,3359.0,430000.0,0.007812,1,2023,1,1
2889,https://www.instagram.com/p/CnDB8PkBfw_/,costacoffee,2023-01-05,37910.0,430000.0,0.088163,1,2023,1,1
2888,https://www.instagram.com/p/CnKGYoEMemU/,costacoffee,2023-01-08,1361.0,430000.0,0.003165,1,2023,1,0
...,...,...,...,...,...,...,...,...,...,...
2463,https://www.instagram.com/p/DHL0oNLKME0/,costacoffee,2025-03-14,703.0,430000.0,0.001635,3,2025,1,-1
2462,https://www.instagram.com/p/DHtQChnKUGt/,costacoffee,2025-03-27,440.0,430000.0,0.001023,3,2025,1,-1
2461,https://www.instagram.com/p/DIT_nTxhSr-/,costacoffee,2025-04-11,501.0,430000.0,0.001165,4,2025,2,-1
2460,https://www.instagram.com/p/DIlOc3utoOc/,costacoffee,2025-04-18,498.0,430000.0,0.001158,4,2025,2,-1


In [12]:
px.line(costa, x="day_posted", y="var", color="q", labels={"var": "Ratio between Likes and Followers",
                                                                    "day_posted": "Date"},
                                                                    hover_data=["post_link", "q"])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([['https://www.instagram.com/p/Cm3XRm5soUL/?img_index=1', 0],
                                   ['https://www.instagram.com/p/CnKGYoEMemU/', 0],
                                   ['https://www.instagram.com/p/CnNG7akh4te/', 0],
                                   ...,
                                   ['https://www.instagram.com/p/DEprmLqNK4w/', 0],
                                   ['https://www.instagram.com/p/DF2Lzg5K2BX/', 0],
                                   ['https://www.instagram.com/p/DIye63iNPc_/', 0]], dtype=object),
              'hovertemplate': ('q=%{customdata[1]}<br>Date=%{x' ... '{customdata[0]}<extra></extra>'),
              'legendgroup': '0',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '0',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2023-01-01T00:00:00', '2023-01-08T00:00:00', '2023-01-09T00:00:00',
                          '2023-01-11T00:00:00', '2023-01-14T00:00:00', '2023-01-16T00:00:00',
                          '2023-01-19T00:00:00', '2023-01-22T00:00:00', '2023-01-23T00:00:00',
                          '2023-01-24T00:00:00', '2023-01-25T00:00:00', '2023-01-27T00:00:00',
                          '2023-01-28T00:00:00', '2023-01-29T00:00:00', '2023-01-30T00:00:00',
                          '2023-02-01T00:00:00', '2023-02-09T00:00:00', '2023-02-10T00:00:00',
                          '2023-02-12T00:00:00', '2023-02-14T00:00:00', '2023-02-16T00:00:00',
                          '2023-02-24T00:00:00', '2023-02-27T00:00:00', '2023-03-07T00:00:00',
                          '2023-03-09T00:00:00', '2023-03-10T00:00:00', '2023-03-13T00:00:00',
                          '2023-03-17T00:00:00', '2023-03-18T00:00:00', '2023-03-26T00:00:00',
                          '2023-03-31T00:00:00', '2023-04-02T00:00:00', '2023-04-04T00:00:00',
                          '2023-04-07T00:00:00', '2023-04-10T00:00:00', '2023-04-13T00:00:00',
                          '2023-04-17T00:00:00', '2023-04-18T00:00:00', '2023-04-20T00:00:00',
                          '2023-04-23T00:00:00', '2023-04-26T00:00:00', '2023-04-30T00:00:00',
                          '2023-05-03T00:00:00', '2023-05-04T00:00:00', '2023-05-06T00:00:00',
                          '2023-05-07T00:00:00', '2023-05-12T00:00:00', '2023-05-18T00:00:00',
                          '2023-05-21T00:00:00', '2023-05-25T00:00:00', '2023-05-27T00:00:00',
                          '2023-06-02T00:00:00', '2023-06-05T00:00:00', '2023-06-09T00:00:00',
                          '2023-06-13T00:00:00', '2023-06-18T00:00:00', '2023-06-19T00:00:00',
                          '2023-06-23T00:00:00', '2023-06-29T00:00:00', '2023-07-06T00:00:00',
                          '2023-07-08T00:00:00', '2023-07-10T00:00:00', '2023-07-11T00:00:00',
                          '2023-07-12T00:00:00', '2023-07-14T00:00:00', '2023-07-16T00:00:00',
                          '2023-07-23T00:00:00', '2023-07-25T00:00:00', '2023-07-28T00:00:00',
                          '2023-07-30T00:00:00', '2023-08-15T00:00:00', '2023-08-23T00:00:00',
                          '2023-08-25T00:00:00', '2023-08-25T00:00:00', '2023-09-03T00:00:00',
                          '2023-09-08T00:00:00', '2023-09-12T00:00:00', '2023-09-17T00:00:00',
                          '2023-09-24T00:00:00', '2023-10-01T00:00:00', '2023-10-02T00:00:00',
                          '2023-10-05T00:00:00', '2023-10-09T00:00:00', '2023-10-13T00:00:00',
                          '2023-10-13T00:00:00', '2023-10-23T00:00:00', '2023-10-25T00:00:00',
                          '2023-10-25T00:00:00', '2023-10-27T00:00:00', '2023-10-29T00:00:00',
                          '2023-11-07T00:00:00', '2023-11-08T00:00:00', '2023-11-09T00:00:00',
                          '2023-11-11T00:00:00', '2023-11-15T00:00:00'

In [13]:
costa[costa['q']==-1].sort_values(by='var').head(50)

,post_link,cafe,day_posted,n_likes,followers,var,month,year,trimester,q
2886,https://www.instagram.com/p/CnPgbbyhzhR/,costacoffee,2023-01-10,54.0,430000.0,0.000126,1,2023,1,-1
2633,https://www.instagram.com/p/C5lsbEAqt2p/,costacoffee,2024-04-10,177.0,430000.0,0.000412,4,2024,2,-1
2558,https://www.instagram.com/p/C-DhCH6J09l/,costacoffee,2024-07-30,211.0,430000.0,0.000491,7,2024,3,-1
2560,https://www.instagram.com/p/C-C8rWMKFN8/,costacoffee,2024-07-30,222.0,430000.0,0.000516,7,2024,3,-1
2588,https://www.instagram.com/p/C8e_KQ8K2qT/,costacoffee,2024-06-21,228.0,430000.0,0.000530,6,2024,2,-1
2553,https://www.instagram.com/p/C-VMMRrKeXa/,costacoffee,2024-08-06,235.0,430000.0,0.000547,8,2024,3,-1
2489,https://www.instagram.com/p/DDdE55NuThI/,costacoffee,2024-12-11,238.0,430000.0,0.000553,12,2024,4,-1
2634,https://www.instagram.com/p/C5gQ8GhKMi9/,costacoffee,2024-04-08,258.0,430000.0,0.000600,4,2024,2,-1
2547,https://www.instagram.com/p/C-28d1cKfi3/,costacoffee,2024-08-19,258.0,430000.0,0.000600,8,2024,3,-1
2641,https://www.instagram.com/p/C5GFfJbKtHy/,costacoffee,2024-03-29,260.0,430000.0,0.000605,3,2024,1,-1
